In [ ]:
# String methods

In [ ]:
import pandas as pd

In [ ]:
# Series and Index are equipped with a set of string processing methods that make it easy to operate on each element of the array. Perhaps most importantly, these methods exclude missing/NA values automatically. These are accessed via the str attribute and generally have names matching the equivalent (scalar) built-in string methods:

In [ ]:
s = pd.Series(
    ["A", "B", "C", "Aaba", "Baca", np.nan, "CABA", "dog", "cat"], dtype="string"
)


In [ ]:
s.str.lower()

In [ ]:
idx = pd.Index([" jack", "jill ", " jesse ", "frank"])

In [ ]:
idx.str.strip()

In [ ]:
# The string methods on Index are especially useful for cleaning up or transforming DataFrame columns. For instance, you may have columns with leading or trailing whitespace:

In [ ]:
df = pd.DataFrame(
    np.random.randn(3, 2), columns=[" Column A ", " Column B "], index=range(3))


In [ ]:
# Since df.columns is an Index object, we can use the .str accessor

In [ ]:
df.columns.str.strip()

In [ ]:
df.columns.str.lower()

In [ ]:
# These string methods can then be used to clean up the columns as needed. Here we are removing leading and trailing whitespaces, lower casing all names, and replacing any remaining whitespaces with underscores:

In [ ]:
df.columns = df.columns.str.strip().str.lower().str.replace(" ", "_")

In [ ]:
# If you have a Series where lots of elements are repeated (i.e. the number of unique elements in the Series is a lot smaller than the length of the Series), it can be faster to convert the original Series to one of type category and then use .str.<method> or .dt.<property> on that. The performance difference comes from the fact that, for Series of type category, the string operations are done on the .categories and not on each element of the Series.

# Please note that a Series of type category with string .categories has some limitations in comparison to Series of type string (e.g. you can’t add strings to each other: s + " " + s won’t work if s is a Series of type category). Also, .str methods which operate on elements of type list are not available on such a Series.

# Splitting and replacing strings

In [ ]:
s2 = pd.Series(["a_b_c", "c_d_e", np.nan, "f_g_h"], dtype="string")
s2.str.split("_")

In [ ]:
# Elements in the split lists can be accessed using get or [] notation:
s2.str.split("_").str.get(1)

In [ ]:
s2.str.split("_").str[1]

In [ ]:
# It is easy to expand this to return a DataFrame using expand.
s2.str.split("_", expand=True)

In [ ]:
# When original Series has StringDtype, the output columns will all be StringDtype as well.
# It is also possible to limit the number of splits:

In [ ]:
s2.str.split("_", expand=True, n=1)

In [ ]:
# rsplit is similar to split except it works in the reverse direction, i.e., from the end of the string to the beginning of the string:

In [ ]:
s2.str.rsplit("_", expand=True, n=1)

In [ ]:
# replace optionally uses regular expressions:
s3 = pd.Series(
    ["A", "B", "C", "Aaba", "Baca", "", np.nan, "CABA", "dog", "cat"],
    dtype="string",
)

In [ ]:
s3.str.replace("^.a|dog", "XX-XX ", case=False, regex=True)

In [ ]:
# Some caution must be taken when dealing with regular expressions! The current behavior is to treat single character patterns as literal strings, even when regex is set to True. This behavior is deprecated and will be removed in a future version so that the regex keyword is always respected.

In [ ]:
# If you want literal replacement of a string (equivalent to str.replace()), you can set the optional regex parameter to False, rather than escaping each character. In this case both pat and repl must be strings:

In [ ]:
# f you want literal replacement of a string (equivalent to str.replace()), you can set the optional regex parameter to False, rather than escaping each character. In this case both pat and repl must be strings:

In [ ]:
dollars = pd.Series(["12", "-$10", "$10,000"], dtype="string")
dollars.str.replace(r"-\$", "-", regex=True)
dollars.str.replace("-$", "-", regex=False)

In [ ]:
# The replace method can also take a callable as replacement. It is called on every pat using re.sub(). The callable should expect one positional argument (a regex object) and return a string.

In [ ]:
#Reverse every lowercase alphabetic word
pat = r"[a-z]+"

def repl(m):
    return m.group(0)[::-1]

In [ ]:
pd.Series(["foo 123", "bar baz", np.nan], dtype="string").str.replace(
    pat, repl, regex=True
)

In [ ]:
pat = r"(?P<one>\w+) (?P<two>\w+) (?P<three>\w+)"

def repl(m):
    return m.group("two").swapcase()


pd.Series(["Foo Bar Baz", np.nan], dtype="string").str.replace(
    pat, repl, regex=True
)

In [ ]:
# The replace method also accepts a compiled regular expression object from re.compile() as a pattern. All flags should be included in the compiled regular expression object.

In [ ]:
import re
regex_pat = re.compile(r"^.a|dog", flags=re.IGNORECASE)
s3.str.replace(regex_pat, "XX-XX ", regex=True)

In [ ]:
# Including a flags argument when calling replace with a compiled regular expression object will raise a ValueError.

In [ ]:
s = pd.Series(["str_foo", "str_bar", "no_prefix"])
s.str.removeprefix("str_")

In [ ]:
s = pd.Series(["foo_str", "bar_str", "no_suffix"])

s.str.removesuffix("_str")

## Extracting substrings

In [ ]:
# The extract method accepts a regular expression with at least one capture group.

# Extracting a regular expression with more than one group returns a DataFrame with one column per group.

In [ ]:
pd.Series(
    ["a1", "b2", "c3"],
    dtype="string",
).str.extract(r"([ab])(\d)", expand=False)


In [ ]:
# Elements that do not match return a row filled with NaN. Thus, a Series of messy strings can be “converted” into a like-indexed Series or DataFrame of cleaned-up or more useful strings, without necessitating get() to access tuples or re.match objects. The dtype of the result is always object, even if no match is found and the result only contains NaN.

In [2]:
pd.Series(["a1", "b2", "c3"], dtype="string").str.extract(
    r"(?P<letter>[ab])(?P<digit>\d)", expand=False
)

NameError: name 'pd' is not defined

In [ ]:
pd.Series(
    ["a1", "b2", "3"],
    dtype="string",
).str.extract(r"([ab])?(\d)", expand=False)


In [ ]:
# can also be used. Note that any capture group names in the regular expression will be used for column names; otherwise capture group numbers will be used.

# Extracting a regular expression with one group returns a DataFrame with one column if

In [ ]:
pd.Series(["a1", "b2", "c3"], dtype="string").str.extract(r"[ab](\d)", expand=True)
# It returns a Series if expand=False.
pd.Series(["a1", "b2", "c3"], dtype="string").str.extract(r"[ab](\d)", expand=False)


## Extract all matches in each subject (extractall)

In [ ]:
Unlike extract (which returns only the first match),


In [ ]:
s = pd.Series(["a1", "b2", "c3"], ["A11", "B22", "C33"], dtype="string")

s

In [ ]:
# the extractall method returns every match. The result of extractall is always a DataFrame with a MultiIndex on its rows. The last level of the MultiIndex is named match and indicates the order in the subject.

In [ ]:
s.str.extractall(two_groups)
s = pd.Series(["a3", "b3", "c2"], dtype="string")
# then extractall(pat).xs(0, level='match') gives the same result as extract(pat).

In [ ]:
s.index.str.extract("(?P<letter>[a-zA-Z])", expand=True)
# It returns an Index if expand=False.
s.index.str.extract("(?P<letter>[a-zA-Z])", expand=False)
# Calling on an Index with a regex with more than one capture group returns a DataFrame if expand=True.
s.index.str.extract("(?P<letter>[a-zA-Z])([0-9]+)", expand=True)

In [ ]:
s = pd.Series(["a1a2", "b1", "c1"], index=["A", "B", "C"], dtype="string")
two_groups = "(?P<letter>[a-z])(?P<digit>[0-9])"
s.str.extract(two_groups, expand=True)

In [ ]:
# the extractall method returns every match. The result of extractall is always a DataFrame with a MultiIndex on its rows. The last level of the MultiIndex is named match and indicates the order in the subject.
s.str.extractall(two_groups)

In [ ]:
# when each subject string in the Series has exactly one match,
s = pd.Series(["a3", "b3", "c2"], dtype="string")

In [ ]:
extract_result = s.str.extract(two_groups, expand=True)
extractall_result = s.str.extractall(two_groups)
extractall_result.xs(0, level="match")

In [ ]:
# Index also supports .str.extractall. It returns a DataFrame which has the same result as a Series.str.extractall with a default index (starts from 0).

In [ ]:
pd.Index(["a1a2", "b1", "c1"]).str.extractall(two_groups)
pd.Series(["a1a2", "b1", "c1"], dtype="string").str.extractall(two_groups)

## Testing for strings that match or contain a pattern

In [ ]:
#   You can check whether elements contain a pattern:

In [ ]:
pattern = r"[0-9][a-z]"
pd.Series(
    ["1", "2", "3a", "3b", "03c", "4dx"],
    dtype="string",
).str.contains(pattern)

In [ ]:
pd.Series(
    ["1", "2", "3a", "3b", "03c", "4dx"],
    dtype="string",
).str.match(pattern)

In [ ]:
pd.Series(
    ["1", "2", "3a", "3b", "03c", "4dx"],
    dtype="string",
).str.fullmatch(pattern)

<!-- The distinction between match, fullmatch, and contains is strictness: fullmatch tests whether the entire string matches the regular expression; match tests whether there is a match of the regular expression that begins at the first character of the string; and contains tests whether there is a match of the regular expression at any position within the string.

The corresponding functions in the re package for these three match modes are re.fullmatch, re.match, and re.search, respectively.Methods like match, fullmatch, contains, startswith, and endswith take an extra na argument so missing values can be considered True or False: -->

In [ ]:
s4 = pd.Series(
    ["A", "B", "C", "Aaba", "Baca", np.nan, "CABA", "dog", "cat"], dtype="string"
)
s4.str.contains("A", na=False)

## Concatenation

In [ ]:
# There are several ways to concatenate a Series or Index, either with itself or others, all based on cat(), resp. Index.str.cat.

In [ ]:
# Concatenating a single Series into a string. The content of a Series (or Index) can be concatenated:

In [ ]:
s = pd.Series(["a", "b", "c", "d"], dtype="string")
s.str.cat(sep=",")

In [ ]:
# By default, missing values are ignored. Using na_rep, they can be given a representation:

In [ ]:
t = pd.Series(["a", "b", np.nan, "d"], dtype="string")
t.str.cat(sep=",")
t.str.cat(sep=",", na_rep="-")

# Concatenating a Series and something list-like into a Series

In [ ]:
# The first argument to cat() can be a list-like object, provided that it matches the length of the calling Series (or Index).

In [1]:
s.str.cat(["A", "B", "C", "D"])

NameError: name 's' is not defined

In [ ]:
# Missing values on either side will result in missing values in the result as well, unless na_rep is specified:

In [ ]:
s.str.cat(t)
s.str.cat(t, na_rep="-")

In [ ]:
# he first argument to cat() can be a list-like object, provided that it matches the length of the calling Series (or Index).

In [ ]:
s.str.cat(["A", "B", "C", "D"])

In [ ]:
Missing values on either side will result in missing values in the result as well, unless na_rep is specified

In [ ]:
s.str.cat(t)

In [ ]:
s.str.cat(t, na_rep="-")

## Concatenating a Series and something array-like into a Series -->

In [ ]:
# The parameter others can also be two-dimensional. In this case, the number or rows must match the lengths of the calling Series (or Index).

In [ ]:
d = pd.concat([t, s], axis=1)

In [ ]:

s.str.cat(d, na_rep="-")

## Concatenating a Series and an indexed object into a Series, with alignment -->

In [ ]:
# For concatenation with a Series or DataFrame, it is possible to align the indexes before concatenation by setting the join-keyword.

In [ ]:
u = pd.Series(["b", "d", "a", "c"], index=[1, 3, 0, 2], dtype="string")

In [ ]:
s.str.cat(u)
s.str.cat(u, join="left")

In [ ]:
# The usual options are available for join (one of 'left', 'outer', 'inner', 'right'). In particular, alignment also means that the different lengths do not need to coincide anymore.

In [ ]:
# If the join keyword is not passed, the method cat() will currently fall back to the behavior before version 0.23.0 (i.e. no alignment), but a FutureWarning will be raised if any of the involved indexes differ, since this default will change to join='left' in a future version.## Concatenating a Series and an indexed object into a Series, with alignment -->

In [ ]:
v = pd.Series(["z", "a", "b", "d", "e"], index=[-1, 0, 1, 3, 4], dtype="string")

In [ ]:
s.str.cat(v, join="left", na_rep="-")

s.str.cat(v, join="outer", na_rep="-")

In [ ]:
# If the join keyword is not passed, the method cat() will currently fall back to the behavior before version 0.23.0 (i.e. no alignment), but a FutureWarning will be raised if any of the involved indexes differ, since this default will change to join='left' in a future version.

In [ ]:
# The same alignment can be used when others is a DataFrame:

In [ ]:
f = d.loc[[3, 2, 1, 0], :]
s.str.cat(f, join="left", na_rep="-")

## Concatenating a Series and many objects into a Series

In [ ]:
# Several array-like items (specifically: Series, Index, and 1-dimensional variants of np.ndarray) can be combined in a list-like container (including iterators, dict-views, etc.).

In [ ]:
s.str.cat([u, u.to_numpy()], join="left")

In [ ]:
# All elements without an index (e.g. np.ndarray) within the passed list-like must match in length to the calling Series (or Index), but Series and Index may have arbitrary length (as long as alignment is not disabled with join=None):

In [ ]:
s.str.cat([v, u, u.to_numpy()], join="outer", na_rep="-")

In [ ]:
# If using join='right' on a list-like of others that contains different indexes, the union of these indexes will be used as the basis for the final concatenation:

In [ ]:
s.str.cat([u.loc[[3]], v.loc[[-1, 0]]], join="right", na_rep="-")

## Creating indicator variables

In [ ]:
s = pd.Series(["a", "a|b", np.nan, "a|c"], dtype="string")
s.str.get_dummies(sep="|")

## Indexing with str

In [ ]:
# You can use [] notation to directly index by position locations. If you index past the end of the string, the result will be a NaN.

In [ ]:
# String Index also supports get_dummies which returns a MultiIndex.

In [ ]:
idx = pd.Index(["a", "a|b", np.nan, "a|c"])

idx.str.get_dummies(sep="|")

In [ ]:
# Method summary
# Method

# Description

# cat()

# Concatenate strings

# split()

# Split strings on delimiter

# rsplit()

# Split strings on delimiter working from the end of the string

# get()

# Index into each element (retrieve i-th element)

# join()

# Join strings in each element of the Series with passed separator

# get_dummies()

# Split strings on the delimiter returning DataFrame of dummy variables

# contains()

# Return boolean array if each string contains pattern/regex

# replace()

# Replace occurrences of pattern/regex/string with some other string or the return value of a callable given the occurrence

# removeprefix()

# Remove prefix from string, i.e. only remove if string starts with prefix.

# removesuffix()

# Remove suffix from string, i.e. only remove if string ends with suffix.

# repeat()

# Duplicate values (s.str.repeat(3) equivalent to x * 3)

# pad()

# Add whitespace to left, right, or both sides of strings

# center()

# Equivalent to str.center

# ljust()

# Equivalent to str.ljust

# rjust()

# Equivalent to str.rjust

# zfill()

# Equivalent to str.zfill

# wrap()

# Split long strings into lines with length less than a given width

# slice()

# Slice each string in the Series

# slice_replace()

# Replace slice in each string with passed value

# count()

# Count occurrences of pattern

# startswith()

# Equivalent to str.startswith(pat) for each element

# endswith()

# Equivalent to str.endswith(pat) for each element

# findall()

# Compute list of all occurrences of pattern/regex for each string

# match()

# Call re.match on each element, returning matched groups as list

# extract()

# Call re.search on each element, returning DataFrame with one row for each element and one column for each regex capture group

# extractall()

# Call re.findall on each element, returning DataFrame with one row for each match and one column for each regex capture group

# len()

# Compute string lengths

# strip()

# Equivalent to str.strip

# rstrip()

# Equivalent to str.rstrip

# lstrip()

# Equivalent to str.lstrip

# partition()

# Equivalent to str.partition

# rpartition()

# Equivalent to str.rpartition

# lower()

# Equivalent to str.lower

# casefold()

# Equivalent to str.casefold

# upper()

# Equivalent to str.upper

# find()

# Equivalent to str.find

# rfind()

# Equivalent to str.rfind

# index()

# Equivalent to str.index

# rindex()

# Equivalent to str.rindex

# capitalize()

# Equivalent to str.capitalize

# swapcase()

# Equivalent to str.swapcase

# normalize()

# Return Unicode normal form. Equivalent to unicodedata.normalize

# translate()

# Equivalent to str.translate

# isalnum()

# Equivalent to str.isalnum

# isalpha()

# Equivalent to str.isalpha

# isdigit()

# Equivalent to str.isdigit

# isspace()

# Equivalent to str.isspace

# islower()

# Equivalent to str.islower

# isupper()

# Equivalent to str.isupper

# istitle()

# Equivalent to str.istitle

# isnumeric()

# Equivalent to str.isnumeric

# isdecimal()

# Equivalent to str.isdecimal

In [ ]:
s = pd.Series(
    ["A", "B", "C", "Aaba", "Baca", np.nan, "CABA", "dog", "cat"], dtype="string"
)


s.str[0]

In [ ]:
s.str[1]

In [ ]:
# String Index also supports get_dummies which returns a MultiIndex.

In [ ]:
idx = pd.Index(["a", "a|b", np.nan, "a|c"])
idx.str.get_dummies(sep="|")